In [ ]:
!git clone https://github.com/tensorflow/tpu.git

Cloning into 'tpu'...
remote: Enumerating objects: 10287, done.
remote: Counting objects: 100% (489/489), done.
remote: Compressing objects: 100% (299/299), done.
remote: Total 10287 (delta 257), reused 344 (delta 182), pack-reused 9798
Receiving objects: 100% (10287/10287), 24.52 MiB | 30.50 MiB/s, done.
Resolving deltas: 100% (7295/7295), done.


In [ ]:
%cd tpu/models/official/resnet/

/content/tpu/models/official/resnet


In [ ]:
### COPY AND PASTE CONTENT FROM REPO

!touch resnet_main_v2.py

In [ ]:
import tensorflow as tf
import os
import numpy as np
import subprocess

In [ ]:
# Download image for inference

PICTURE = 'panda.jpg'
!wget https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG -O {PICTURE}

--2021-10-17 06:52:43--  https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116068 (113K) [image/jpeg]
Saving to: ‘panda.jpg’

panda.jpg           100%[===================>] 113.35K  --.-KB/s    in 0.03s   

2021-10-17 06:52:44 (3.39 MB/s) - ‘panda.jpg’ saved [116068/116068]



In [ ]:
MODELS = [
    {"name": "resnetrs50_i160", "tar_name": "resnet-rs-50-i160.tar.gz", "config_name":"resnetrs50_i160.yaml"},
    {"name": "resnetrs101_i160", "tar_name": "resnet-rs-101-i160.tar.gz", "config_name":"resnetrs101_i160.yaml"},
    {"name": "resnetrs101_i192", "tar_name": "resnet-rs-101-i192.tar.gz", "config_name":"resnetrs101_i192.yaml"},
    {"name": "resnetrs152_i192", "tar_name": "resnet-rs-152-i192.tar.gz", "config_name":"resnetrs152_i192.yaml"},
    {"name": "resnetrs152_i224", "tar_name": "resnet-rs-152-i224.tar.gz", "config_name":"resnetrs152_i224.yaml"},
    {"name": "resnetrs350_i320", "tar_name": "resnet-rs-350-i320.tar.gz", "config_name":"resnetrs350_i320.yaml"},
    {"name": "resnetrs350_i256", "tar_name": "resnet-rs-350-i256.tar.gz", "config_name":"resnetrs350_i256.yaml"},
    {"name": "resnetrs270_i256", "tar_name": "resnet-rs-270-i256.tar.gz", "config_name":"resnetrs270_i256.yaml"},
    {"name": "resnetrs152_i256", "tar_name": "resnet-rs-152-i256.tar.gz", "config_name":"resnetrs152_i256.yaml"},
    {"name": "resnetrs200_i256", "tar_name": "resnet-rs-200-i256.tar.gz", "config_name":"resnetrs200_i256.yaml"},
    {"name": "resnetrs420_i320", "tar_name": "resnet-rs-420-i320.tar.gz", "config_name":"resnetrs420_i320.yaml"},
]


In [ ]:
!rm /content/*.npy

rm: cannot remove '/content/*.npy': No such file or directory


In [ ]:
!rm -r resnetrs*

rm: cannot remove 'resnetrs*': No such file or directory


In [ ]:
for model_params in MODELS:

    # Create model_dir
    model_name = model_params["name"]
    !mkdir $model_name

    # Download config file
    config_name = model_params["config_name"]
    !cd $model_name; wget https://raw.githubusercontent.com/tensorflow/tpu/master/models/official/resnet/resnet_rs/configs/$config_name

    # Download weights
    tar_name = model_params["tar_name"]
    !curl https://storage.googleapis.com/cloud-tpu-checkpoints/resnet-rs/$tar_name | tar xzv -C $model_name
    
    # Create saved_model dir inside model_dir
    export_dir = os.path.join(model_name, "saved_model")
    !mkdir $export_dir


    if "checkpoint" not in os.listdir(model_name):
        lines = ['model_checkpoint_path: "model.ckpt"', 'all_model_checkpoint_paths: "model.ckpt"']
        with open(f"{model_name}/checkpoint", "w") as f:
            f.writelines(lines)

    # Create command for export:
    config_path = os.path.join(model_name, config_name)
    cmd = f"""
export PYTHONPATH=$PYTHONPATH:/content/tpu/models && python resnet_main_v2.py \
     --tpu=mock \
     --model_dir={model_name} \
     --mode=predict \
     --export_dir={export_dir} \
     --config_file={config_path} \
     --export_moving_average
 """

    # Run export    
    subprocess.run(cmd, shell=True)
    # !export PYTHONPATH=$PYTHONPATH:/content/tpu/models && python resnet_main_v2.py \
    #      --tpu=mock \
    #      --model_dir=resnetrs50_i160 \
    #      --mode=predict \
    #      --export_dir=resnetrs50_i160/saved_model \
    #      --config_file=resnetrs50_i160/resnetrs152_i256.yaml \
    #      --export_moving_average


    # Load exported saved model
    model_number = os.listdir(export_dir)[0]
    saved_model_dir = os.path.join(export_dir, model_number)
    loaded = tf.saved_model.load(saved_model_dir)

    # Load array for inference
    input_tensor = tf.io.read_file(PICTURE)

    # Run inference
    infer = loaded.signatures["serving_default"]  
    out = infer(tf.constant(input_tensor[None,...]))
    
    logits = out["logits"].numpy()
    inputs = out["features"].numpy()
    
    # Save output arrays
    output_name = f"/content/{model_name}_original_logits.npy"
    np.save(output_name, logits)


--2021-10-17 07:02:07--  https://raw.githubusercontent.com/tensorflow/tpu/master/models/official/resnet/resnet_rs/configs/resnetrs50_i160.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 580 [text/plain]
Saving to: ‘resnetrs50_i160.yaml’

resnetrs50_i160.yam 100%[===================>]     580  --.-KB/s    in 0s      

2021-10-17 07:02:07 (31.2 MB/s) - ‘resnetrs50_i160.yaml’ saved [580/580]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0checkpoint
model.ckpt.data-00000-of-00001
 93  381M   93  357M    0     0  55.7M      0  0:00:06  0:00:06 --:--:-- 55.9Mmodel.ckpt.ind

In [ ]:
%cd /content/

/content


In [ ]:
!ls *.npy | wc -l

11


In [ ]:
!zip -r resnet-rs-original-outputs-ema.zip *.npy

  adding: resnetrs101_i160_original_logits.npy (deflated 7%)
  adding: resnetrs101_i192_original_logits.npy (deflated 7%)
  adding: resnetrs152_i192_original_logits.npy (deflated 7%)
  adding: resnetrs152_i224_original_logits.npy (deflated 7%)
  adding: resnetrs152_i256_original_logits.npy (deflated 7%)
  adding: resnetrs200_i256_original_logits.npy (deflated 7%)
  adding: resnetrs270_i256_original_logits.npy (deflated 7%)
  adding: resnetrs350_i256_original_logits.npy (deflated 7%)
  adding: resnetrs350_i320_original_logits.npy (deflated 7%)
  adding: resnetrs420_i320_original_logits.npy (deflated 7%)
  adding: resnetrs50_i160_original_logits.npy (deflated 7%)


In [ ]:
from google.colab import files

In [ ]:
files.download("resnet-rs-original-outputs-ema.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>